In [147]:
#При необходимости устанавливаем 
#!pip install vk_api

In [131]:
import vk_api
import csv

In [133]:
def scrape_wall_data(token, owner_id, count=10):
    """
    Функция принимает на вход три аргумента: 
    1. token - токен для подключения к ВК по API;
    2. owner_id - ID страницы, с которой берутся данные;
    3. count - объем итоговой выборки.
    
    Функция возвращает ID поста, ссылку на изображение, прикрепленное к посту, и количество лайков на посте.
    
    """
    
    try:
        # Подключаемся к VK через API
        vk_session = vk_api.VkApi(token=token)
        vk = vk_session.get_api()

        # Создаем хранилище для данных
        scraped_data = []

        # Достаем посты со страницы с помощью метода wall.get
        posts = vk.wall.get(owner_id=owner_id, count=count)['items']
        
        ## Собираем необходимую информацию, парсинг
        # Достаем ссылку на изображение
        for post in posts: 
            if 'attachments' in post:
                for attachment in post['attachments']:
                    if attachment['type'] == 'photo':
                        photo_sizes = attachment['photo']['sizes']
                        # Получаем самую большую версию фото
                        largest_photo = max(photo_sizes, key=lambda x: x['width'] * x['height'])
                        photo_url = largest_photo['url']

        # Собираем все данные в одно место
        for post in posts:
            post_data = {
                'post_id': post['id'], 
                'post_photo_link': photo_url,
                'post_likes': post['likes']['count']
            }
            scraped_data.append(post_data)

        return scraped_data

    except vk_api.exceptions.ApiError as e:
        print(f"Error accessing VK API: {e}")
        return []

In [149]:
# Функция для сохранения *.csv
def save_to_csv(data, file_name='vk_wall_posts_hype.csv'):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['post_id', 'post_photo_link', 'post_likes'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [153]:
# Функция для выполнения задачи
def main():
    # Токен VK API
    token = "vk1.a.pDR6c5BFvhxrUTNwgl630lGtko_5ch5pozVv2bj0p8b5bLBpcGyBlPpiVNM3YTbVzas_47xwoh3VSXg6EGR-O5aki2WwgOviq85-4XweSgzXkfXKqwQeho8p6PKTBlY8_Gsx5EDYjNPswmL-nfV24_rydTRCV3G1bEeyNG6fXqcQ33xgMef4sktyUWsPXi3UbpVVprnPj9A-iHDj0OVkcw"

    # ID сообщества
    owner_id = -187853043

    # Количество публикаций для скрейпинга
    num_posts = 100

    # Запуск функции для скрейпинга
    wall_data = scrape_wall_data(token, owner_id, num_posts)

    save_to_csv(wall_data)

if __name__ == "__main__":
    main()